In [162]:
from bs4 import BeautifulSoup
import requests
import re
from spacy.lang.en import English
from spacy.lang.en.stop_words import STOP_WORDS
import string
import spacy
nlp = spacy.load('en')
import pandas as pd

#### Get Article Text

In [230]:
# LETTER FROM BIRMINGHAM JAIL

#html_doc = requests.get('https://www.theatlantic.com/magazine/archive/2018/02/letter-from-a-birmingham-jail/552461/')
soup = BeautifulSoup(html_doc.text, 'html.parser')

sections = soup.find_all('section', id=re.compile("^article"))
paragraphs = [item.get_text() for sublist in sections for item in sublist]
paragraphs = paragraphs[6:-2] # Drops bits of Atlantic text

corpus = ' '.join(paragraphs)

#print(corpus)

In [284]:
# I HAVE A DREAM

headers = {"User-Agent":'Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36'}
html_doc = requests.get('https://www.americanrhetoric.com/speeches/mlkihaveadream.htm',
                        headers=headers)
soup = BeautifulSoup(html_doc.text, 'html.parser')

paragraphs = soup.find_all('font', attrs={'face': 'Verdana', 'size': '2'})
paragraphs = [x.get_text() for x in paragraphs]
paragraphs = [x.replace('\r', ' ').replace('\t', '').replace('\n', '').replace('\xa0', '') for x in paragraphs]

# <p align="left" style="line-height: 150%"><font face="Verdana" size="2">I am happy to join with you today in what will go down in history as


print(paragraphs)
#print(soup)

['I am happy to join with you today in what will go down in history as the greatest demonstration for freedom in the history of our nation.', 'Five score years ago,  a great American, in whose symbolic shadow we stand today, signed the  Emancipation Proclamation. This momentous decree came as a great beacon light of hope to millions of Negro slaves who had been seared in the flames of withering injustice. It came as a joyous daybreak to end the long night of their captivity.', "But one hundred years later, the Negro still is not free. One hundred years later, the life of the Negro is still sadly crippled by the manacles of segregation and the chains of discrimination. One hundred years later, the Negro lives on a lonely island of poverty in the midst of a vast ocean of material prosperity. One hundred years later, the Negro is still languished in the corners of American society and finds himself an exile in his own land. And so we've come here today to dramatize a shameful condition.",

In [205]:
# THE CASE FOR REPARATIONS

#html_doc = requests.get('https://www.theatlantic.com/magazine/archive/2014/06/the-case-for-reparations/361631/')
soup = BeautifulSoup(html_doc.text, 'html.parser')

sections = soup.find_all('section', id=re.compile("^article"))
paragraphs = [x.find_all('p') for x in sections]
paragraphs = [item.get_text() for sublist in paragraphs for item in sublist]
paragraphs.remove('Listen to the audio version of this article:Feature stories, read aloud: download the Audm app for your iPhone.')

corpus = ' '.join(paragraphs)

#print(corpus)

#### Remove Stop Words

In [211]:
def remove_stop_words(corpus):

    original_doc = nlp.make_doc(corpus)
    filtered_doc = []

    for token in original_doc:

        word = token.text.lower()
        if (word in STOP_WORDS or word in string.punctuation
                or word in ('“', '”', '—')):
            continue
        else:
            filtered_doc.append(token)

    #print(filtered_doc)
    
    return filtered_doc

#### Lemmatization

In [212]:
def lemmatization(filtered_doc):

    lemmas = []

    for token in filtered_doc:
        lemmas.append(token.lemma_)

    #print(lemmas)
    
    return lemmas

#### Sentiment Dictionary

In [213]:
sentiment_df = pd.read_csv('subjectivity_clues_hltemnlp05/subjclueslen1-HLTEMNLP05.tff', 
                                 sep=" |=",
                                 names=['junk', 'type', 'junk2', 'len', 'junk3', 'word1', 
                                        'junk4', 'pos1', 'junk5', 'stemmed1', 'junk6', 'priorpolarity'],
                                 usecols=['type', 'len', 'word1', 'pos1', 'stemmed1', 'priorpolarity'],
                                 error_bad_lines=False
                                )

sentiment_dict = pd.Series(sentiment_df.priorpolarity.values,index=sentiment_df.word1).to_dict()

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


#### Sentiment Analysis

In [214]:
def sentiment_analysis(lemmas):

    results = {'positive': 0, 'negative': 0, 'both': 0, 'neutral': 0, 'not_in_dict': 0}

    for lemma in lemmas:
        lemma = lemma.lower()
        if lemma in sentiment_dict:
            results[sentiment_dict[lemma]] += 1
    #        print (lemma, sentiment_dict[lemma])
        else:
            results['not_in_dict'] += 1

    return results


In [218]:
for text in paragraphs:

    filtered_doc = remove_stop_words(text)
    lemmas = lemmatization(filtered_doc)
    results = sentiment_analysis(lemmas)

    print(results)

{'positive': 0, 'negative': 0, 'both': 0, 'neutral': 0, 'not_in_dict': 2}
{'positive': 0, 'negative': 0, 'both': 0, 'neutral': 0, 'not_in_dict': 12}
{'positive': 0, 'negative': 0, 'both': 0, 'neutral': 0, 'not_in_dict': 2}
{'positive': 5, 'negative': 5, 'both': 0, 'neutral': 0, 'not_in_dict': 44}
{'positive': 2, 'negative': 3, 'both': 0, 'neutral': 1, 'not_in_dict': 19}
{'positive': 2, 'negative': 2, 'both': 0, 'neutral': 1, 'not_in_dict': 37}
{'positive': 1, 'negative': 8, 'both': 0, 'neutral': 2, 'not_in_dict': 42}
{'positive': 1, 'negative': 2, 'both': 0, 'neutral': 0, 'not_in_dict': 30}
{'positive': 0, 'negative': 1, 'both': 0, 'neutral': 0, 'not_in_dict': 8}
{'positive': 2, 'negative': 1, 'both': 0, 'neutral': 5, 'not_in_dict': 37}
{'positive': 2, 'negative': 5, 'both': 0, 'neutral': 1, 'not_in_dict': 41}
{'positive': 1, 'negative': 0, 'both': 0, 'neutral': 0, 'not_in_dict': 8}
{'positive': 5, 'negative': 5, 'both': 0, 'neutral': 2, 'not_in_dict': 40}
{'positive': 3, 'negative': 0